# Cleaning

In [140]:
import numpy as np
import pandas as pd

from pathlib import Path
import re

In this file I will:

1. Remove non-informative/unuseful columns.
1. Convert the format of columns so they can be used in my model or feature engineering.
1. Drop the outliers in the price column.
1. Fill missing values.
1. Relabel categorical data with a small amount of data in certain categories.
1. Drop columns with missing values that can't be filled.

In [141]:
# Read in the data
RAW_PATH = Path('../data/raw/')
listings = pd.read_csv(RAW_PATH/'listings.csv')

## Drop Non-informative/Unuseful Columns

In [142]:
# List of keywords contained in non-informative columns
keywords = ['host', 'scrape', 'url', 'review', 'availability', 'calendar', 'license', 'require', 'country']
non_informative_cols = [
    {column for column in listings.columns if keyword in column}
    for keyword in keywords 
]
non_inform_cols = {
    'id',
    'city',
    'state',
    'zipcode',
    'street',
    'market',
    'smart_location',
    'name',
    'experiences_offered',
    'is_business_travel_ready',
    'security_deposit',
    'jurisdiction_names',
    'neighbourhood',
    'weekly_price',
    'monthly_price'
}
non_informative_cols = set().union(*non_informative_cols, non_inform_cols)
listings = listings.drop(non_informative_cols, axis=1)

In [143]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22552 entries, 0 to 22551
Data columns (total 31 columns):
summary                         21589 non-null object
space                           14020 non-null object
description                     22349 non-null object
neighborhood_overview           11540 non-null object
notes                           7215 non-null object
transit                         13036 non-null object
access                          10837 non-null object
interaction                     10406 non-null object
house_rules                     11449 non-null object
neighbourhood_cleansed          22552 non-null object
neighbourhood_group_cleansed    22552 non-null object
latitude                        22552 non-null float64
longitude                       22552 non-null float64
is_location_exact               22552 non-null object
property_type                   22552 non-null object
room_type                       22552 non-null object
accommodates              

## Convert Format of Columns

We'll start by changing any currency columns to floats.

In [144]:
def price_to_float(price):
    """
    Removes all charcters not digits or decimal points and converts to float.
    """
    if price is np.nan:
        return np.nan
    return float(re.sub('[^\d.]', '', price))

In [145]:
currency_cols = ['price', 'cleaning_fee', 'extra_people']
for col in currency_cols:
    listings[col] = listings[col].apply(price_to_float)

Now we will store the amenities as a set instead of a string.

In [146]:
listings['amenities'] = listings['amenities'].apply(lambda x: set(re.sub('[{}"]', '', x).split(',')))

Now we will convert any boolean columns to binary, 1 being true and 0 being false.

In [147]:
binary_cols = ['is_location_exact', 'instant_bookable']
for col in binary_cols:
    listings[col] = listings[col].map({'f': 0, 't': 1})

For the cancellation policy it makes sense to encode the column using discrete values as the data is ordinal.

In [148]:
listings['cancellation_policy'].value_counts()

flexible                       9102
moderate                       7096
strict_14_with_grace_period    6287
super_strict_30                  53
super_strict_60                  14
Name: cancellation_policy, dtype: int64

There is a clear order to the data so it can be encoded as follows.

In [149]:
cancellation_order = {
    'flexible': 1,
    'moderate': 2,
    'strict_14_with_grace_period': 3,
    'super_strict_30': 4,
    'super_strict_60': 5,
}
listings['cancellation_policy'] = listings['cancellation_policy'].map(cancellation_order)

## Drop Outliers From Price Column

I have decided to drop any listings with a price greater than the mean of the prices + 2 standard deviations and also any listings that have a price of $0.

In [150]:
price_ulim = listings['price'].mean() + 2*listings['price'].std()
listings = listings[(listings['price'] <= price_ulim) & (listings['price'] > 0)]

## Fill Missing Values

In [151]:
listings.isnull().sum()[listings.isnull().sum() != 0]

summary                    937
space                     8515
description                202
neighborhood_overview    10966
notes                    15284
transit                   9473
access                   11665
interaction              12121
house_rules              11077
bathrooms                   32
bedrooms                    18
beds                        38
square_feet              22026
cleaning_fee              7098
dtype: int64

I will fill values in bathrooms, bedrooms, beds, cleaning fee using information from the other columns. The other columns will be used in feature engineering and so can be left with missing values.

### Bedrooms

Let's look at the listings that are missing bedrooms information to see what we can learn.

In [152]:
listings[listings['bedrooms'].isnull()][['property_type', 'room_type', 'accommodates']]

,property_type,room_type,accommodates
10,Apartment,Entire home/apt,2
1396,Apartment,Entire home/apt,2
1608,Apartment,Entire home/apt,2
2235,Apartment,Entire home/apt,2
5118,Apartment,Entire home/apt,2
5203,Apartment,Entire home/apt,2
6197,Apartment,Entire home/apt,3
8257,Apartment,Entire home/apt,4
8294,Apartment,Entire home/apt,3
8638,Condominium,Entire home/apt,3


In [153]:
listings.corr()['bedrooms'].sort_values(ascending=False)[1:]

accommodates           0.626890
beds                   0.606399
square_feet            0.486248
price                  0.468868
guests_included        0.384301
cleaning_fee           0.310133
bathrooms              0.303755
extra_people           0.137417
cancellation_policy    0.119857
is_location_exact      0.020709
latitude               0.008599
minimum_nights         0.005595
maximum_nights        -0.002645
longitude             -0.005355
instant_bookable      -0.013201
Name: bedrooms, dtype: float64

It makes sense to use the accommodates column to fill the missing values, as they are the most highly correlated.

In [154]:
listings.groupby('accommodates')['bedrooms'].agg(pd.Series.mode)

accommodates
1              1
2              1
3              1
4              1
5              2
6              2
7              3
8              3
9              2
10             3
11             5
12             3
13    [3.0, 4.0]
14             5
15             4
16             5
Name: bedrooms, dtype: object

All of our listings with missing bedrooms values have accommodates values between 1 and 4. The most common number of bedrooms for listings the accomodate 1 to 4 is 1 bedroom for all. It therefore makes sense to fill all the missing bedroom values with 1.

In [155]:
listings['bedrooms'] = listings['bedrooms'].fillna(1)

### Beds

In [156]:
listings.corr()['beds'].sort_values(ascending=False)[1:]

accommodates           0.792433
bedrooms               0.606370
price                  0.452463
guests_included        0.444171
square_feet            0.387882
cleaning_fee           0.294383
bathrooms              0.255530
extra_people           0.196813
cancellation_policy    0.162115
is_location_exact      0.051145
instant_bookable       0.049816
latitude               0.012542
minimum_nights         0.008693
maximum_nights        -0.005655
longitude             -0.011015
Name: beds, dtype: float64

There is a very high correlation between accomodates and beds, therefore it makes sense to fill these values in the same way we did above.

In [157]:
beds_mode = listings.groupby('accommodates')['beds'].agg(pd.Series.mode)
listings['beds'] = listings.apply(
    lambda x: np.nan_to_num(x['beds'], nan=beds_mode[x['accommodates']].mean()), axis=1
)

### Bathrooms

In [158]:
listings.corr()['bathrooms'].sort_values(ascending=False)[1:]

square_feet            0.347321
bedrooms               0.303766
beds                   0.255417
price                  0.251163
accommodates           0.243170
guests_included        0.176524
cleaning_fee           0.161011
extra_people           0.058756
is_location_exact      0.034665
cancellation_policy    0.031344
instant_bookable       0.012190
latitude              -0.001963
maximum_nights        -0.002993
minimum_nights        -0.005207
longitude             -0.008316
Name: bathrooms, dtype: float64

None of the other features are very highly correlated with bathrooms. Let's look at how the number of bathrooms are distributed.

In [159]:
listings['bathrooms'].value_counts()

1.0    19758
2.0     1251
1.5     1144
0.5      102
2.5       71
3.0       48
0.0       41
4.0       10
8.5        3
7.5        2
3.5        2
5.0        2
8.0        2
7.0        2
4.5        1
6.0        1
Name: bathrooms, dtype: int64

The most common value by some distance is 1, it makes sense to fill any missing values with this.

In [160]:
listings['bathrooms'] = listings['bathrooms'].fillna(1)

### Cleaning Fee

For missing cleaning fees I will assume it is either missing because it is zero or is missing because it is included in the price. So will fill any missing values with 0.

In [161]:
listings['cleaning_fee'] = listings['cleaning_fee'].fillna(0)

## Relabeling Categorical Data

### Property Type

First lets combine similar property types.

In [162]:
property_map = {
    'Townhouse': "House",
    'Guesthouse': "House",
    'Villa': "House",
    'Bungalow': "House",
    'Cottage': "House",
    'Boutique hotel': "Hotel",
    'Aparthotel': "Hotel",
}
listings['property_type'] = listings['property_type'].apply(lambda x: property_map.get(x, x))

Now we will group any property types containing less than 50 listings to be a 'Other'.

In [163]:
other_properties = (listings['property_type'].value_counts() < 50)
listings['property_type'] = listings['property_type'].apply(lambda x: np.where(other_properties[x], 'Other', x))

Now we have cleaned the data we can save it as a csv for feature engineering.

In [164]:
PROCESSED_PATH = Path('../data/processed/')
listings.to_csv(PROCESSED_PATH/'cleaned_listings.csv', index=False)